# Notebook for running ERDiff alignment step by step

In [1]:
from torch.autograd import Variable
from torch.utils.data import DataLoader

import numpy as np
import scipy.signal as signal
from sklearn.metrics import r2_score
from tqdm import tqdm_notebook

import pickle
import random

import sys
sys.path.append("..")
from model_functions.Diffusion import *
from model_functions.VAE import *
from model_functions.ERDiff_utils import *
from model_functions.MLA_Model import *
from model_functions.VAE_Readout import *

## 1. Source Domain: Training

Extract spatio-temporal structure for source domain  
Code for training is `VAE_Diffusion_CoTrain.py`

### 1.1 Load source domain data

- trial spikes: neural firing rates
- trial vel: velocity
- trial dir: labels of direction, 8 classes

In [2]:
# Specify the trial length and number of neurons to use
len_trial,num_neurons = 37, 187

In [3]:
with open('../datasets/Neural_Source.pkl', 'rb') as f:
    train_data1 = pickle.load(f)['data']
train_trial_spikes1, train_trial_vel1, train_trial_dir1 = train_data1['firing_rates'], train_data1['velocity'], train_data1['labels']

There are 209 trials, for each trial, the time length to use is 37 (minimal trial length - 1), number of neurons is 187, dimension of velocity is 2.

In [4]:
print("minimal trial length is:", min(train_trial_spikes1[i].shape[0] for i in range(len(train_trial_spikes1))))

minimal trial length is: 38


In [5]:
start_pos = 1 

train_trial_spikes_tide1 = np.array([spike[start_pos:len_trial+start_pos, :num_neurons] for spike in train_trial_spikes1])
print("Shape for training neural signal:", np.shape(train_trial_spikes_tide1))

train_trial_vel_tide1 = np.array([spike[start_pos:len_trial+start_pos, :] for spike in train_trial_vel1])
print("Shape for training velocity:", np.shape(train_trial_vel_tide1))

train_trial_spikes_tide = train_trial_spikes_tide1
train_trial_vel_tide = train_trial_vel_tide1

Shape for training neural signal: (209, 37, 187)
Shape for training velocity: (209, 37, 2)


In [6]:
array_train_trial_dir1 = np.expand_dims(np.array((train_trial_dir1), dtype=object),1)

train_trial_spikes_tide = train_trial_spikes_tide1
train_trial_vel_tide = train_trial_vel_tide1
train_trial_dic_tide = np.squeeze(np.vstack([array_train_trial_dir1]))
print(np.shape(train_trial_dic_tide))

(209,)


### 1.2 Data preprocessing

Apply gaussian smoother  

$$
w(n)=\exp^{-\frac{1}{2}(\frac{n}{\sigma})^2}
$$

`scipy.signal.windows.gaussian(M, std, sym=True)`
```
Parameters:	
- M : int, number of points in the output window. If zero or less, an empty array is returned.
- std : float, the standard deviation, sigma.
- sym : bool, optional, when True (default), generates a symmetric window, for use in filter design. When False, generates a periodic window, for use in spectral analysis.

Returns:	
- w : ndarray, The window, with the maximum value normalized to 1 (though the value 1 does not appear if M is even and sym is True).
```

In [7]:
bin_width = float(0.02) * 1000 # /ms

kern_sd_ms = 100
kern_sd = int(round(kern_sd_ms / bin_width))
print("M = std =", kern_sd)
window = signal.windows.gaussian(kern_sd, kern_sd, sym=True)
window /= np.sum(window)
print("window:", window)

M = std = 5
window: [0.19205063 0.20392638 0.20804597 0.20392638 0.19205063]


Apply convolution filter  

$$(v1 * v2) [n] = \sum v1[m] v2[n-m]$$

In [8]:
filt = lambda x: np.convolve(x, window, 'same')
train_trial_spikes_smoothed = np.apply_along_axis(filt, 1, train_trial_spikes_tide)

### 1.3 Randomly shuffle and split to train and validation set

In [9]:
indices = np.arange(train_trial_spikes_tide.shape[0])
np.random.seed(2023) 
np.random.shuffle(indices)
train_len = round(len(indices) * 0.80)
real_train_trial_spikes_smed, val_trial_spikes_smed = train_trial_spikes_smoothed[indices[:train_len]], train_trial_spikes_smoothed[indices[train_len:]]
real_train_trial_vel_tide, val_trial_vel_tide = train_trial_vel_tide[indices[:train_len]], train_trial_vel_tide[indices[train_len:]]
real_train_trial_dic_tide, val_trial_dic_tide = train_trial_dic_tide[indices[:train_len]], train_trial_dic_tide[indices[train_len:]]

In [10]:
real_train_trial_spikes_stand = (real_train_trial_spikes_smed)
val_trial_spikes_stand = (val_trial_spikes_smed)

spike_train = Variable(torch.from_numpy(real_train_trial_spikes_stand)).float()
spike_val = Variable(torch.from_numpy(val_trial_spikes_stand)).float()

emg_train = Variable(torch.from_numpy(real_train_trial_vel_tide)).float()
emg_val = Variable(torch.from_numpy(val_trial_vel_tide)).float()

### 1.4 Experiment settings

In [11]:
n_epochs = 500
batch_size = 16
global_batch_size = 16
ae_res_weight = 10 # scale for spike reconstruction loss
kld_weight = 1 # scale for KL-Divergence loss
n_batches = len(real_train_trial_spikes_smed)//batch_size
print("number of batches:", n_batches)

mse_criterion = nn.MSELoss()
poisson_criterion = nn.PoissonNLLLoss(log_input=False)

l_rate = 0.001 # learning rate
timesteps = 50

number of batches: 10


### 1.5 Build the model


In [12]:
# Loss function, AE reconstruction loss, emg loss, kl-divergence loss
def get_loss(model, spike, emg):
    re_sp_, vel_hat_, mu, log_var = model(spike, train_flag= True)
    ae_loss = poisson_criterion(re_sp_, spike)
    emg_loss = mse_criterion(vel_hat_, emg)
    kld_loss = torch.mean(0.5 * (- log_var + mu ** 2 + log_var.exp() - 1))
    total_loss = ae_res_weight * ae_loss + emg_loss + kld_weight * kld_loss
    return total_loss

In [13]:
# Training
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
setup_seed(21)

In [14]:
# VAE model
model = VAE_Model()
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

input_dim = 1

dm_model = diff_STBlock(input_dim)
dm_model.to(device)

dm_optimizer = Adam(dm_model.parameters(), lr=1e-3)

/home/cheny/anaconda3/envs/torch/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


### 1.6 Train

In [16]:
pre_loss = 1e10 # Save the loss
pre_total_loss_ = 1e18
vae_loss_list = []
diff_loss_list = []

In [17]:
for epoch in tqdm_notebook(range(n_epochs)):
    spike_gen_obj = get_batches(real_train_trial_spikes_stand,batch_size)
    emg_gen_obj = get_batches(real_train_trial_vel_tide,batch_size)
    for ii in range(n_batches):
        optimizer.zero_grad()
        spike_batch = next(spike_gen_obj)
        emg_batch = next(emg_gen_obj)

        spike_batch = Variable(torch.from_numpy(spike_batch)).float()
        emg_batch = Variable(torch.from_numpy(emg_batch)).float()

        # Loss
        batch_loss = get_loss(model, spike_batch, emg_batch)

        batch_loss.backward()
        optimizer.step()
        

    with torch.no_grad():
        val_total_loss = get_loss(model, spike_val, emg_val)
        vae_loss_list.append(val_total_loss.item())

        _, _, train_latents, _ = model(spike_train, train_flag = False)

        if val_total_loss < pre_total_loss_: 
            pre_total_loss_ = val_total_loss
            torch.save(model.state_dict(),'source_vae_model')

            np.save("train_latents.npy",train_latents)

        
    train_latents = np.expand_dims(train_latents,1).astype(np.float32)
    train_spike_data = train_latents.transpose(0,1,3,2)

    dataloader = DataLoader(train_spike_data, batch_size=global_batch_size)

    total_loss = 0
    for step, batch in enumerate(dataloader):
        dm_optimizer.zero_grad()

        batch_size = batch.shape[0]
        batch = batch.to(device)

        # generate diffusion timesteps (noise scale) for each batch from range
        t = torch.randint(0, timesteps, (batch_size,), device=device).long()

        loss = p_losses(dm_model, batch, t)

        print("Step", step, " Loss:", loss.item())
        total_loss += loss.item()

        loss.backward()
        dm_optimizer.step()

    print("total Loss of epoch ", epoch+1, " is ", total_loss)
    diff_loss_list.append(total_loss)

    if total_loss < pre_loss:
        pre_loss = total_loss
        torch.save(dm_model.state_dict(), 'source_diffusion_model')

/tmp/ipykernel_1408/643933650.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(n_epochs)):


  0%|          | 0/500 [00:00<?, ?it/s]

Step 0  Loss: 0.8186498284339905
Step 1  Loss: 0.8044883608818054
Step 2  Loss: 0.8157080411911011
Step 3  Loss: 0.7985531687736511
Step 4  Loss: 0.7848435640335083
Step 5  Loss: 0.7880468368530273
Step 6  Loss: 0.7691790461540222
Step 7  Loss: 0.7987245321273804
Step 8  Loss: 0.7530821561813354
Step 9  Loss: 0.7631310224533081
Step 10  Loss: 0.7303891777992249
total Loss of epoch  1  is  8.624795734882355
Step 0  Loss: 0.7383076548576355
Step 1  Loss: 0.7348331809043884
Step 2  Loss: 0.7234946489334106
Step 3  Loss: 0.7092326879501343
Step 4  Loss: 0.7228999733924866
Step 5  Loss: 0.7175029516220093
Step 6  Loss: 0.6981369256973267
Step 7  Loss: 0.6672934889793396
Step 8  Loss: 0.6532304883003235
Step 9  Loss: 0.6326757669448853
Step 10  Loss: 0.6543505787849426
total Loss of epoch  2  is  7.651958346366882
Step 0  Loss: 0.6528767943382263
Step 1  Loss: 0.5653676986694336
Step 2  Loss: 0.613252580165863
Step 3  Loss: 0.5598500370979309
Step 4  Loss: 0.6057943105697632
Step 5  Loss: 0.

## 2. Target domain: Maximum Likelihood Alignment

### 2.1 Load the data and preparing (same as in source domain)

In [18]:
len_trial, num_neurons_s, num_neurons_t = 37, 187, 172

In [19]:
with open('../datasets/Neural_Target.pkl', 'rb') as f:
    test_data = pickle.load(f)['data']
test_trial_spikes, test_trial_vel, test_trial_dir = test_data['firing_rates'], test_data['velocity'], np.squeeze(test_data['labels'])

In [20]:
test_trial_spikes_tide = np.array([spike[:len_trial, :num_neurons_t] for spike in test_trial_spikes])
print("Shape for test neural signal:", np.shape(test_trial_spikes_tide))

test_trial_vel_tide = np.array([spike[:len_trial, :] for spike in test_trial_vel])
print("Shape for test velocity:",np.shape(test_trial_vel_tide))

Shape for test neural signal: (184, 37, 172)
Shape for test velocity: (184, 37, 2)


In [21]:
test_trial_spikes_smoothed = np.apply_along_axis(filt, 1, test_trial_spikes_tide)
test_trial_spikes_stand = (test_trial_spikes_smoothed)
spike_test = Variable(torch.from_numpy(test_trial_spikes_stand)).float()

In [22]:
test_trial_spikes_stand_half_len = len(test_trial_spikes_stand) // 2

spike_day_0 = Variable(torch.from_numpy(real_train_trial_spikes_stand)).float()
spike_day_k = Variable(torch.from_numpy(test_trial_spikes_stand[:test_trial_spikes_stand_half_len])).float()

num_x, num_y, num_y_test = spike_day_0.shape[0], spike_day_k.shape[0], test_trial_spikes_stand.shape[0]

p = Variable(torch.from_numpy(np.full((num_x, 1), 1 / num_x))).float()
q = Variable(torch.from_numpy(np.full((num_y, 1), 1 / num_y))).float()
q_test = Variable(torch.from_numpy(np.full((num_y_test, 1), 1 / num_y_test))).float()

### 2.2 Define hyperparameters

In [23]:
n_epochs = 500
epoches = 300
batch_size = 64
timesteps = 50
eps = 1 / timesteps
channels = 1
input_dim = 1
pre_total_loss_ = 1e18
l_rate = 1e-3

### 2.3 Load the model

In [24]:
def logger_performance(model):
    re_sp_test, vel_hat_test, _, _, _, _,_ = model(spike_train, spike_test, p, q_test, train_flag=False)

    sys.stdout.flush()
    # print(test_trial_vel_tide.reshape((-1,2)))
    # print(vel_hat_test.reshape((-1,2)))
    key_metric = 100 * r2_score(test_trial_vel_tide.reshape((-1,2)),vel_hat_test.reshape((-1,2)), multioutput='uniform_average')
    return  key_metric

In [25]:
diff_model = diff_STBlock(input_dim)
diff_model_dict = torch.load('source_diffusion_model')
diff_model.load_state_dict(diff_model_dict)
for k,v in diff_model.named_parameters():
    v.requires_grad=False

In [58]:
setup_seed(21)
# vanilla_model_dict = torch.load('../model_checkpoints_old/source_vae_model')
vanilla_model_dict = torch.load('source_vae_model')
MLA_model = VAE_MLA_Model()
MLA_dict_keys = MLA_model.state_dict().keys()
vanilla_model_dict_keys = vanilla_model_dict.keys()

MLA_dict_new = MLA_model.state_dict().copy()

for key in vanilla_model_dict_keys:
    MLA_dict_new[key] = vanilla_model_dict[key]

MLA_model.load_state_dict(MLA_dict_new)

optimizer_mla = torch.optim.Adam(MLA_model.parameters(), lr=l_rate)
criterion = nn.MSELoss()
poisson_criterion = nn.PoissonNLLLoss(log_input=False)

for param in MLA_model.vde_rnn.parameters():
    param.requires_grad = False

for param in MLA_model.sde_rnn.parameters():
    param.requires_grad = False
    
for param in MLA_model.encoder_rnn.parameters():
    param.requires_grad = False

MLA_model.low_d_readin_s.weight.requires_grad = False
MLA_model.low_d_readin_s.bias.requires_grad = False
MLA_model.fc_mu_1.weight.requires_grad = False
MLA_model.fc_mu_1.bias.requires_grad = False
MLA_model.fc_log_var_1.weight.requires_grad = False
MLA_model.fc_log_var_1.bias.requires_grad = False
MLA_model.sde_fc1.weight.requires_grad = False
MLA_model.sde_fc1.bias.requires_grad = False
MLA_model.sde_fc2.weight.requires_grad = False
MLA_model.sde_fc2.bias.requires_grad = False
MLA_model.vde_fc_minus_0.weight.requires_grad = False

### 2.4 Maximum Likelihood Alignment

In [59]:
key_metric = -1000
appro_alpha = 1.2

In [60]:
optimizer_mla.zero_grad()
re_sp, _, distri_0, distri_k, latents_k, output_sh_loss, log_var = MLA_model(spike_day_0, spike_day_k, p, q, train_flag=True)

In [61]:
output_sh_loss.item()

0.4189572334289551

In [62]:
latents_k = latents_k[:, None, :, :]
latents_k = torch.transpose(latents_k,3,2)
print(latents_k.shape)

torch.Size([92, 1, 8, 37])


In [63]:
batch_size = latents_k.shape[0]
t = torch.randint(0, timesteps, (batch_size,), device="cpu").long()
noise = torch.randn_like(latents_k)

z_noisy = q_sample(x_start=latents_k, t=t, noise=noise)
predicted_noise = diff_model(z_noisy, t)

In [64]:
noise_loss = F.smooth_l1_loss(noise, predicted_noise)

In [65]:
noise_loss.item()

0.1560746729373932

In [66]:
skill_div = skilling_divergence(z_noisy,latents_k,t)

In [67]:
skill_div.item()

0.0006736140348948538

In [68]:
total_loss = output_sh_loss + appro_alpha * noise_loss + skill_div

In [69]:
total_loss.item()

0.6069204211235046

In [70]:
total_loss += 0.001

In [71]:
print("Before update")
print("="*50)
for name, parms in MLA_model.named_parameters():
    print('-->name', name)
    print('-->para', parms)
    print('-->grad_requires', parms.requires_grad)
    print('-->grad_value', parms.grad)
    print("="*50)

Before update
-->name low_d_readin_s.weight
-->para Parameter containing:
tensor([[-0.0039, -0.2162, -0.1601,  ...,  0.0373, -0.0459,  0.0731],
        [ 0.0006,  0.1748, -0.2171,  ..., -0.0218,  0.0857, -0.0485],
        [ 0.0568, -0.3316, -0.0619,  ...,  0.0599,  0.1428, -0.0464],
        ...,
        [-0.0119,  0.2882,  0.1063,  ...,  0.0553,  0.1437,  0.0525],
        [ 0.0071, -0.1596, -0.0646,  ..., -0.0429, -0.0746,  0.0046],
        [-0.0015, -0.2137,  0.3009,  ...,  0.0314, -0.0358, -0.0514]])
-->grad_requires False
-->grad_value None
-->name low_d_readin_s.bias
-->para Parameter containing:
tensor([-0.0516, -0.0206,  0.0231, -0.0717,  0.0721, -0.0335, -0.0278,  0.0027,
        -0.0643,  0.0031,  0.0318,  0.0148, -0.0155, -0.0365, -0.0099,  0.0119,
         0.0318,  0.0140, -0.0493, -0.0264,  0.0728,  0.0486,  0.0071, -0.0132,
         0.0428, -0.0333,  0.0057, -0.0414, -0.0076, -0.0301, -0.0214,  0.0436,
        -0.0710,  0.0352, -0.0286, -0.0365,  0.0070, -0.0453, -0.0547,  

In [72]:
# total_loss.backward(retain_graph=True)
total_loss.backward()

In [73]:
print("After backward")
print("="*50)
for name, parms in MLA_model.named_parameters():
    print('-->name', name)
    # print('-->para', parms)
    print('-->grad_requires', parms.requires_grad)
    print('-->grad_value', parms.grad)
    print("="*50)

After backward
-->name low_d_readin_s.weight
-->grad_requires False
-->grad_value None
-->name low_d_readin_s.bias
-->grad_requires False
-->grad_value None
-->name align_layer.weight
-->grad_requires True
-->grad_value None
-->name low_d_readin_t.weight
-->grad_requires True
-->grad_value tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
-->name low_d_readin_t.bias
-->grad_requires True
-->grad_value tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
-->name en

In [74]:
print(MLA_model.low_d_readin_t.weight.grad)

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])


In [57]:
total_loss_list = []
for epoch in range(epoches):

    optimizer_mla.zero_grad()

    re_sp, _, distri_0, distri_k, latents_k, output_sh_loss, log_var = MLA_model(spike_day_0, spike_day_k, p, q, train_flag=True)

    total_loss = output_sh_loss

    latents_k = latents_k[:, None, :, :]
    latents_k = torch.transpose(latents_k,3,2)

    batch_size = latents_k.shape[0]
    t = torch.randint(0, timesteps, (batch_size,), device="cpu").long()
    noise = torch.randn_like(latents_k)

    z_noisy = q_sample(x_start=latents_k, t=t, noise=noise)
    predicted_noise = diff_model(z_noisy, t)
    total_loss += appro_alpha * F.smooth_l1_loss(noise, predicted_noise)
    
    total_loss += skilling_divergence(z_noisy,latents_k,t)

    total_loss.backward(retain_graph=True)
    optimizer_mla.step()

    total_loss_list.append(total_loss.item())

    with torch.no_grad():
        if (epoch % 5 == 0) or (epoch == n_epochs-1):
            print(f"epoch={epoch}, total loss is: {total_loss.item()}")
            current_metric = float(logger_performance(MLA_model))
            if current_metric > key_metric:
                key_metric = current_metric
            if total_loss < pre_total_loss_:
                torch.save(MLA_model.state_dict(),'vae_model_mla')
                pre_total_loss_ = total_loss 

epoch=0, total loss is: 0.6069204211235046


/home/cheny/anaconda3/envs/torch/lib/python3.11/site-packages/ot/bregman/_sinkhorn.py:506: UserWarning: Warning: numerical errors at iteration 0
  warnings.warn('Warning: numerical errors at iteration %d' % ii)


ValueError: Input contains NaN.